In [1]:
from citylearn_wrapper import CityLearnEnvWrapper
import multiprocessing as mp
import numpy as np
import pandas as pd
import os
from rbc_agent import RBCAgent
from constants import *

/home/doseok/miniconda3/envs/ActiveRL/lib/python3.10/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


In [2]:
paths = [f"data/Test_{env_name}" for env_name in CL_ENV_KEYS]
schemas = [os.path.join(cl_path, "schema.json") for cl_path in paths]
env_configs = [{
    "schema": schema,
    "is_evaluation": True,
} for schema in schemas]

In [3]:
envs = [CityLearnEnvWrapper(env_config) for env_config in env_configs]
def pool_fn(input):
    i, env = input
    obs = env.reset()
    done = False
    obss = []
    next_obss = []
    actions = []
    rews = []
    dones = []
    rbc_agent = RBCAgent(env.action_space)
    while not done:
        action = rbc_agent.compute_action(obs)
        next_obs, rew, done, info = env.step(action)
        actions.append(action)
        rews.append(rew)
        obs = next_obs
    return rews, i

TRAIN ENV SCHEMA:  ['/mnt/7076984276980B4C/ssd_backup/ActiveRL/data/Test_cold_Texas']
TRAIN ENV SCHEMA:  ['/mnt/7076984276980B4C/ssd_backup/ActiveRL/data/Test_dry_Cali']
TRAIN ENV SCHEMA:  ['/mnt/7076984276980B4C/ssd_backup/ActiveRL/data/Test_hot_new_york']
TRAIN ENV SCHEMA:  ['/mnt/7076984276980B4C/ssd_backup/ActiveRL/data/Test_snowy_Cali_winter']


In [4]:
def store_data(data, path):
    df = pd.Series(data)
    print(df.sum())
    df.to_csv(os.path.join(path, "rbc_rews.csv"), mode='a', index=False, header=False)

def collect_dataset(envs, num_processes=4):
    obss = []
    next_obss = []
    actions = []
    rews = []
    dones = []
    
    pool = mp.Pool(processes=num_processes)
    results = pool.map(pool_fn, enumerate(envs))
    for result, i in results:
        print(i)
        store_data(result, paths[i])

In [5]:
collect_dataset(envs, 4)

0
-13531.602310933922
1
-10615.03498861454
2
-9864.91520918176
3
-12578.293095695566
